<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.5: Putting it all Together: An FIR Filter
**Prev: [Sequential Logic](2.4_sequential_logic.ipynb)**<br>
**Next: [ChiselTest (was chisel-testers2)](2.6_chiseltest.ipynb)**

## Motivation
이제 Chisel의 기본 사항을 배웠으므로 이 지식을 사용하여 FIR(유한 임펄스 응답) 필터 모듈을 구축해 보겠습니다! FIR 필터는 디지털 신호 처리 애플리케이션에서 매우 일반적입니다. 또한 FIR 필터는 모듈 3에서 자주 다시 나타나므로 건너뛰어서 이 모듈을 걸러내지 않는 것이 중요합니다! FIR 필터에 익숙하지 않다면 [신뢰할 수 있는 Wikipedia](https://en.wikipedia.org/wiki/Finite_impulse_response)의 문서로 이동하여 자세히 알아보세요.

## Setup

In [9]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/parkdongho/dev/chisel-bootcamp/source/load-ivy.sc"

In [10]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

---
# FIR Filter

설계할 FIR 필터는 다음 작업을 수행합니다.

<img src="images/fir.jpg" width="720">
기본적으로 이것은 필터 계수의 요소와 입력 신호의 요소를 요소별로 곱하고 합(_convolution_이라고도 함)을 출력합니다.

또는 신호 정의:

$y[n] = b_0 x[n] + b_1 x[n-1] + b_2 x[n-2] + ...$
- $y[n]$는 $n$ 시간의 출력 신호입니다.
  - $x[n]$는 입력 신호입니다.
  - $b_i$는 필터 계수 또는 임펄스 응답입니다.
  - $n-1$, $n-2$, ...는 시간 $n$이 1, 2, ... 사이클 지연됩니다.
 
## 8-bit Specification

4개의 필터 계수가 매개변수인 4요소 FIR 필터를 작성합니다. module 골격과 기본 테스트가 제공됩니다.
입력과 출력 모두 8비트 부호 없는 정수입니다. 시프트 레지스터와 같은 구성을 사용하여 필요한 상태(예: 지연된 신호 값)를 저장해야 합니다. 제공된 테스터를 사용하여 구현을 확인하십시오.
상수 입력이 있는 레지스터는 시프트 값 1의 `ShiftRegister`를 사용하거나 `RegNext` 구성을 사용하여 만들 수 있습니다.

참고: 테스트를 통과하려면 레지스터를 `0.U`로 초기화해야 합니다.

In [11]:
class My4ElementFir(b0: Int, b1: Int, b2: Int, b3: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })

  ???
}

defined class My4ElementFir

In [ ]:
// Simple sanity check: a element with all zero coefficients should always produce zero
test(new My4ElementFir(0, 0, 0, 0)) { c =>
    c.io.in.poke(0.U)
    c.io.out.expect(0.U)
    c.clock.step(1)
    c.io.in.poke(4.U)
    c.io.out.expect(0.U)
    c.clock.step(1)
    c.io.in.poke(5.U)
    c.io.out.expect(0.U)
    c.clock.step(1)
    c.io.in.poke(2.U)
    c.io.out.expect(0.U)
}

Elaborating design...


In [ ]:
// Simple 4-point moving average
test(new My4ElementFir(1, 1, 1, 1)) { c =>
    c.io.in.poke(1.U)
    c.io.out.expect(1.U)  // 1, 0, 0, 0
    c.clock.step(1)
    c.io.in.poke(4.U)
    c.io.out.expect(5.U)  // 4, 1, 0, 0
    c.clock.step(1)
    c.io.in.poke(3.U)
    c.io.out.expect(8.U)  // 3, 4, 1, 0
    c.clock.step(1)
    c.io.in.poke(2.U)
    c.io.out.expect(10.U)  // 2, 3, 4, 1
    c.clock.step(1)
    c.io.in.poke(7.U)
    c.io.out.expect(16.U)  // 7, 2, 3, 4
    c.clock.step(1)
    c.io.in.poke(0.U)
    c.io.out.expect(12.U)  // 0, 7, 2, 3
}

Elaborating design...


In [ ]:
// Nonsymmetric filter
test(new My4ElementFir(1, 2, 3, 4)) { c =>
    c.io.in.poke(1.U)
    c.io.out.expect(1.U)  // 1*1, 0*2, 0*3, 0*4
    c.clock.step(1)
    c.io.in.poke(4.U)
    c.io.out.expect(6.U)  // 4*1, 1*2, 0*3, 0*4
    c.clock.step(1)
    c.io.in.poke(3.U)
    c.io.out.expect(14.U)  // 3*1, 4*2, 1*3, 0*4
    c.clock.step(1)
    c.io.in.poke(2.U)
    c.io.out.expect(24.U)  // 2*1, 3*2, 4*3, 1*4
    c.clock.step(1)
    c.io.in.poke(7.U)
    c.io.out.expect(36.U)  // 7*1, 2*2, 3*3, 4*4
    c.clock.step(1)
    c.io.in.poke(0.U)
    c.io.out.expect(32.U)  // 0*1, 7*2, 2*3, 3*4
}

Elaborating design...


<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val x_n1 = RegNext(io.in, 0.U)
  val x_n2 = RegNext(x_n1, 0.U)
  val x_n3 = RegNext(x_n2, 0.U)
  io.out := io.in * b0.U(8.W) + 
    x_n1 * b1.U(8.W) +
    x_n2 * b2.U(8.W) + 
    x_n3 * b3.U(8.W)
</pre></article></div></section></div>

---
# FIR Filter Generator

이 모듈에서는 [Module 3.2: Generators: Collection](3.2_collections.ipynb)에서 약간 수정된 예제를 사용할 것입니다.
모듈 3.2를 시작하지 않았더라도 걱정하지 마십시오.
`MyManyDynamicElementVecFir`의 작동 방식에 대한 자세한 내용을 배우게 되지만 기본 아이디어는 FIR 필터 생성기라는 것입니다.

생성기에는 하나의 매개변수인 길이가 있습니다.
이 매개변수는 필터의 탭 수를 나타내며 탭은 하드웨어 `모듈`에 대한 입력입니다.

generator에는 3개의 입력이 있습니다.
* in, 필터에 대한 입력
* valid, 입력이 유효한 때를 나타내는 부울
* consts, 모든 탭에 대한 벡터

및 1개의 출력:
* out, 필터링된 입력

<img src="images/fir.jpg" style="width:450px;"/>

In [15]:
class MyManyDynamicElementVecFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val valid = Input(Bool())
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })
  
  // 이렇게 간결하다니! 이 모든 것이 나중에 무엇을 의미하는지 알게 될 것입니다.
  val taps = Seq(io.in) ++ Seq.fill(io.consts.length - 1)(RegInit(0.U(8.W)))
  taps.zip(taps.tail).foreach { case (a, b) => when (io.valid) { b := a } }

  io.out := taps.zip(io.consts).map { case (a, b) => a * b }.reduce(_ + _)
}

visualize(() => new MyManyDynamicElementVecFir(4))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (0)
 -->
<!-- Title: MyManyDynamicElementVecFir Pages: 1 -->
 
 
 MyManyDynamicElementVecFir 
 
 
 cluster_MyManyDynamicElementVecFir 
 
 MyManyDynamicElementVecFir 
 
<!-- cluster_MyManyDynamicElementVecFir_clock -->
 
 cluster_MyManyDynamicElementVecFir_clock 
 
 clock 
 
<!-- cluster_MyManyDynamicElementVecFir_reset -->
 
 cluster_MyManyDynamicElementVecFir_reset 
 
 reset 
 
<!-- mux_1868417466 -->
 
 mux_1868417466 
 
 
 0 
 
 a? 
 
 ● 
 
 ● 
 
<!-- cluster_MyManyDynamicElementVecFir_reset->mux_1868417466 -->
 
 cluster_MyManyDynamicElementVecFir_reset->mux_1868417466:select 
 
 
 
<!-- mux_703200375 -->
 
 mux_703200375 
 
 
 0 
 
 a? 
 
 ● 
 
 ● 
 
<!-- cluster_MyManyDynamicElementVecFir_reset->mux_703200375 -->
 
 cluster_MyManyDynamicElementVecFir_reset->mux_703200375:select 
 
 
 
<!-- mux_803137972 -->
 
 mux_803137972 
 
 
 0 
 
 a? 
 
 ● 
 
 ● 
 
<!-- cluster_MyManyDynamicElementVecFir_reset->mux_803137972 -->
 
 cluster_MyManyDynamicElementVecFir_reset->mux_803137972:select 
 
 
 
<!-- cluster_MyManyDynamicElementVecFir_io_in -->
 
 cluster_MyManyDynamicElementVecFir_io_in 
 
 io_in 
 
<!-- op_mul_12 -->
 
 op_mul_12 
 
 
 ● 
 
 mul 
 
 ● 
 
 ● 
 
<!-- cluster_MyManyDynamicElementVecFir_io_in->op_mul_12 -->
 
 cluster_MyManyDynamicElementVecFir_io_in->op_mul_12:in1 
 
 
 
<!-- mux_1853659045 -->
 
 mux_1853659045 
 
 
 ● 
 
 a? 
 
 ● 
 
 ● 
 
<!-- cluster_MyManyDynamicElementVecFir_io_in->mux_1853659045 -->
 
 cluster_MyManyDynamicElementVecFir_io_in->mux_1853659045:in1 
 
 
 
<!-- cluster_MyManyDynamicElementVecFir_io_valid -->
 
 cluster_MyManyDynamicElementVecFir_io_valid 
 
 io_valid 
 
<!-- cluster_MyManyDynamicElementVecFir_io_valid->mux_1853659045 -->
 
 cluster_MyManyDynamicElementVecFir_io_valid->mux_1853659045:select 
 
 
 
<!-- mux_1815213218 -->
 
 mux_1815213218 
 
 
 ● 
 
 a? 
 
 ● 
 
 ● 
 
<!-- cluster_MyManyDynamicElementVecFir_io_valid->mux_1815213218 -->
 
 cluster_MyManyDynamicElementVecFir_io_valid->mux_1815213218:select 
 
 
 
<!-- mux_133522559 -->
 
 mux_133522559 
 
 
 ● 
 
 a? 
 
 ● 
 
 ● 
 
<!-- cluster_MyManyDynamicElementVecFir_io_valid->mux_133522559 -->
 
 cluster_MyManyDynamicElementVecFir_io_valid->mux_133522559:select 
 
 
 
<!-- cluster_MyManyDynamicElementVecFir_io_consts_0 -->
 
 cluster_MyManyDynamicElementVecFir_io_consts_0 
 
 io_consts_0 
 
<!-- cluster_MyManyDynamicElementVecFir_io_consts_0->op_mul_12 -->
 
 cluster_MyManyDynamicElementVecFir_io_consts_0->op_mul_12:in2 
 
 
 
<!-- cluster_MyManyDynamicElementVecFir_io_consts_1 -->
 
 cluster_MyManyDynamicElementVecFir_io_consts_1 
 
 io_consts_1 
 
<!-- op_mul_20 -->
 
 op_mul_20 
 
 
 ● 
 
 mul 
 
 ● 
 
 ● 
 
<!-- cluster_MyManyDynamicElementVecFir_io_consts_1->op_mul_20 -->
 
 cluster_MyManyDynamicElementVecFir_io_consts_1->op_mul_20:in2 
 
 
 
<!-- cluster_MyManyDynamicElementVecFir_io_consts_2 -->
 
 cluster_MyManyDynamicElementVecFir_io_consts_2 
 
 io_consts_2 
 
<!-- op_mul_21 -->
 
 op_mul_21 
 
 
 ● 
 
 mul 
 
 ● 
 
 ● 
 
<!-- cluster_MyManyDynamicElementVecFir_io_consts_2->op_mul_21 -->
 
 cluster_MyManyDynamicElementVecFir_io_consts_2->op_mul_21:in2 
 
 
 
<!-- cluster_MyManyDynamicElementVecFir_io_consts_3 -->
 
 cluster_MyManyDynamicElementVecFir_io_consts_3 
 
 io_consts_3 
 
<!-- op_mul_22 -->
 
 op_mul_22 
 
 
 ● 
 
 mul 
 
 ● 
 
 ● 
 
<!-- cluster_MyManyDynamicElementVecFir_io_consts_3->op_mul_22 -->
 
 cluster_MyManyDynamicElementVecFir_io_consts_3->op_mul_22:in2 
 
 
 
<!-- cluster_MyManyDynamicElementVecFir_io_out -->
 
 cluster_MyManyDynamicElementVecFir_io_out 
 
 io_out 
 
<!-- struct_cluster_MyManyDynamicElementVecFir_taps_1 -->
 
 struct_cluster_MyManyDynamicElementVecFir_taps_1 
 
 
 ● 
 
 taps_1 
 
 ● 
 
<!-- struct_cluster_MyManyDynamicElementVecFir_

defined class MyManyDynamicElementVecFir

---
# DspBlock

DSP 구성요소를 더 큰 시스템에 통합하는 것은 어렵고 오류가 발생하기 쉽습니다.
[rocket section of the dsptools repository](https://github.com/ucb-bar/dsptools/tree/master/rocket)은 이러한 작업에 도움이 되는 유용한 생성기로 구성되어 있습니다.

핵심 추상화 중 하나는 `Dsp Block`이라는 개념입니다.
`Dsp Block`에는 다음이 포함됩니다.
* AXI4 Stream 입력 및 출력
* 메모리 매핑된 Status 및 control(본 예에서는 AXI4)

<img src="images/fir_filter.png" style="width:800px;"/>

`DspBlock`은 rocket의 diplomatic 인터페이스를 사용합니다.
[이 사이트](https://www.lowrisc.org/docs/diplomacy/)는 diplomacy의 기본에 대한 좋은 개요를 가지고 있지만 이 예제에서 어떻게 작동하는지 너무 걱정하지 마십시오.
다양한 블록을 연결하여 복잡한 SoC를 형성할 때 diplomacy가 정말 빛을 발합니다.
이 예에서는 단일 주변기기를 만들고 있습니다.
`StandaloneBlock` 특성이 혼합되어 diplomacy 인터페이스가 최상위 IO로 작동합니다.
diplomatic 연결 없이 `DspBlock`이 최상위 인터페이스로 사용될 때만 필요합니다.

다음 코드는 AXI4 인터페이스에서 FIR 필터를 래핑합니다.


In [16]:
import dspblocks._
import freechips.rocketchip.amba.axi4._
import freechips.rocketchip.amba.axi4stream._
import freechips.rocketchip.config._
import freechips.rocketchip.diplomacy._
import freechips.rocketchip.regmapper._

//
// 모든 FIRBlocks의 기본 클래스입니다.
// 이것은 TileLink, AXI4, APB, AHB 등을 FIR 필터의 맛으로 만들기 위해 확장될 수 있습니다.
//
abstract class FIRBlock[D, U, EO, EI, B <: Data](val nFilters: Int, val nTaps: Int)(implicit p: Parameters)
// HasCSR은 메모리 인터페이스가 RegMapper API를 사용하여 상태 및 제어 레지스터를 정의함을 의미합니다.
extends DspBlock[D, U, EO, EI, B] with HasCSR {
    // 스트리밍 인터페이스의 diplomatic node
    // identity 노드는 출력과 입력이 동일하도록 매개변수화됨을 의미합니다.
    val streamNode = AXI4StreamIdentityNode()
    
    // 어떤 하드웨어를 elaborate할지 정의
    lazy val module = new LazyModuleImp(this) {
        // diplomatic node에서 스트리밍 입력 및 출력 와이어를 가져옵니다.
        val (in, _)  = streamNode.in(0)
        val (out, _) = streamNode.out(0)

        require(in.params.n >= nFilters,
                s"""AXI-4 Stream port must be big enough for all 
                   |the filters (need $nFilters,, only have ${in.params.n})""".stripMargin)

        // taps를 저장할 레지스터를 만듭니다.
        val taps = Reg(Vec(nFilters, Vec(nTaps, UInt(8.W))))

        // 메모리 맵 taps과 첫 번째 주소는 필터 레인의 수를 나타내는 읽기 전용 필드입니다.
        val mmap = Seq(
            RegField.r(64, nFilters.U, RegFieldDesc("nFilters", "Number of filter lanes"))
        ) ++ taps.flatMap(_.map(t => RegField(8, t, RegFieldDesc("tap", "Tap"))))

        // 메모리 인터페이스용 하드웨어 생성
        // 이 클래스에서 regmap은 추상(구현되지 않음)입니다. AXI4HasCSR 또는 TLHasCSR과 같은 것을 혼합
        // 특정 메모리 인터페이스에 대한 regmap을 정의합니다.
        regmap(mmap.zipWithIndex.map({case (r, i) => i * 8 -> Seq(r)}): _*)

        // FIR 레인을 만들고 입력과 탭을 연결합니다.
        val outs = for (i <- 0 until nFilters) yield {
            val fir = Module(new MyManyDynamicElementVecFir(nTaps))
            
            fir.io.in := in.bits.data((i+1)*8, i*8)
            fir.io.valid := in.valid && out.ready
            fir.io.consts := taps(i)            
            fir.io.out
        }

        val output = if (outs.length == 1) {
            outs.head
        } else {
            outs.reduce((x: UInt, y: UInt) => Cat(y, x))
        }

        out.bits.data := output
        in.ready  := out.ready
        out.valid := in.valid
    }
}

// FIRBlock의 AXI4 플레이버를 만듭니다.
abstract class AXI4FIRBlock(nFilters: Int, nTaps: Int)(implicit p: Parameters) extends FIRBlock[AXI4MasterPortParameters, AXI4SlavePortParameters, AXI4EdgeParameters, AXI4EdgeParameters, AXI4Bundle](nFilters, nTaps) with AXI4DspBlock with AXI4HasCSR {
    override val mem = Some(AXI4RegisterNode(
        AddressSet(0x0, 0xffffL), beatBytes = 8
    ))
}

// 아래 코드를 실행하면 생성된 firrtl이 표시됩니다.
// LazyModules는 실제로 끌 모듈이 아닙니다. 끌 드라이버를 호출할 때 ".module"을 호출해야 합니다.
// 또한 AXI4StandaloneBlock이 혼합되어 있음을 참고하십시오. 잊어버리면 메모리 때문에 이상한 외교 오류가 발생합니다.
// 인터페이스는 마스터를 예상하고 스트리밍 인터페이스는 연결될 것으로 예상합니다. AXI4StandaloneBlock은 최상위 IO를 추가합니다.
// println(chisel3.Driver.emit(() => LazyModule(new AXI4FIRBlock(1, 8)(Parameters.empty) with AXI4StandaloneBlock).module))

import dspblocks._

import freechips.rocketchip.amba.axi4._

import freechips.rocketchip.amba.axi4stream._

import freechips.rocketchip.config._

import freechips.rocketchip.diplomacy._

import freechips.rocketchip.regmapper._

//
// 모든 FIRBlocks의 기본 클래스입니다.
// 이것은 TileLink, AXI4, APB, AHB 등을 FIR 필터의 맛으로 만들기 위해 확장될 수 있습니다.
//

defined class FIRBlock
defined class AXI4FIRBlock

## Testing

`DspBlock`을 테스트하는 것은 약간 다릅니다.
이제 우리는 메모리 인터페이스와 `LazyModule`을 다루고 있습니다.
dsptools에는 `DspBlock`을 테스트하는 데 도움이 되는 몇 가지 기능이 있습니다.

중요한 기능 중 하나는 `MemMasterModel`입니다.
이 특성은 메모리 트래픽을 생성하기 위한 일반 함수인 `memReadWord` 및 `memWriteWord`와 같은 함수를 정의합니다.
이를 통해 사용 중인 메모리 인터페이스에 특화될 수 있는 하나의 일반 테스트를 작성할 수 있습니다. 예를 들어, 하나의 테스트를 작성한 다음 이를 TileLink 및 AXI4 인터페이스에 대해 특화할 수 있습니다.

아래 코드는 `FIRBlock`을 이런 식으로 테스트합니다.

In [16]:
import dsptools.tester.MemMasterModel
import freechips.rocketchip.amba.axi4

abstract class FIRBlockTester[D, U, EO, EI, B <: Data](c: FIRBlock[D, U, EO, EI, B]) extends PeekPokeTester(c.module) with MemMasterModel {
    // check that address 0 is the number of filters
    require(memReadWord(0) == c.nFilters)
    // write 1 to all the taps
    for (i <- 0 until c.nFilters * c.nTaps) {
        memWriteWord(8 + i * 8, 1)
    }
}

// axi4에 대한 일반 테스터를 전문화합니다.
class AXI4FIRBlockTester(c: AXI4FIRBlock with AXI4StandaloneBlock) extends FIRBlockTester(c) with AXI4MasterModel {
    def memAXI = c.ioMem.get
}

// lazymodules에서 테스터를 호출하는 것은 약간 이상합니다.
// firblocktester는 모듈이 아닌 지연 모듈을 사용합니다("extens PeekPokeTester()"에서 .module을 호출함).
val lm = LazyModule(new AXI4FIRBlock(1, 8)(Parameters.empty) with AXI4StandaloneBlock)
chisel3.iotesters.Driver(() => lm.module) { _ => new AXI4FIRBlockTester(lm) }

cmd16.sc:4: not found: type PeekPokeTester
abstract class FIRBlockTester[D, U, EO, EI, B <: Data](c: FIRBlock[D, U, EO, EI, B]) extends PeekPokeTester(c.module) with MemMasterModel {
                                                                                             ^cmd16.sc:21: type mismatch;
 found   : Helper.this.AXI4FIRBlockTester
 required: chisel3.iotesters.PeekPokeTester[freechips.rocketchip.diplomacy.LazyModuleImp{val in: freechips.rocketchip.amba.axi4stream.AXI4StreamBundle; val out: freechips.rocketchip.amba.axi4stream.AXI4StreamBundle; val taps: chisel3.Vec[chisel3.Vec[chisel3.UInt]]; val mmap: Seq[freechips.rocketchip.regmapper.RegField]; val outs: scala.collection.immutable.IndexedSeq[chisel3.UInt]; val output: chisel3.UInt}]
val res16_5 = chisel3.iotesters.Driver(() => lm.module) { _ => new AXI4FIRBlockTester(lm) }
                                                               ^cmd16.sc:4: no arguments allowed for nullary constructor Object: ()Object
abstract cl

: 

<span style="color:red">**Exercise: TileLink**</span><br>

메모리 상호 연결에 TileLink를 사용하는 `FIRBlock` 버전을 추가하고 TileLink를 사용하도록 `FIRBlockTester`를 확장합니다.

---
# You're done!

[Return to the top.](#top)